In [47]:
import sys
import os
import json

# Add the project root to sys.path to import src
# Adjust this path depending on where you run the notebook
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '23127011'))
if project_root not in sys.path:
    sys.path.append(project_root)

print(f"Added to path: {project_root}")

from src.parser import find_root_tex_file, LatexFlattener, LatexStructureBuilder, LatexContentProcessor
from src.processing import ContentDeduplicator

Added to path: d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\23127011


In [48]:
!pip install bibtexparser texsoup

In [49]:
import os
import re
import bibtexparser
from bibtexparser.bparser import BibTexParser
from TexSoup import TexSoup

class ReferenceProcessor:
    def __init__(self, paper_id, version, root_dir):
        self.paper_id = paper_id
        self.version = version
        self.root_dir = root_dir
        
        # Dùng Dictionary thay vì List để tự động khử trùng lặp theo Key khi parse nhiều nguồn
        self.raw_refs = {} 
        
        # --- REGEX CẤU HÌNH CHO BIBITEM ---
        # Lookahead (?=...) hoạt động hoàn hảo nhờ kỹ thuật Sentinel thêm vào khi xử lý
        self.REGEX_BIBITEM = re.compile(
            r'\\bibitem\s*(?:\[(.*?)\])?\s*\{(.*?)\}\s*(.*?)(?=(?:\\bibitem)|(?:\\end\{thebibliography\})|\Z)', 
            re.DOTALL
        )

    def process_references(self, flat_content):
        """
        Hàm chính: Giữ nguyên signature cũ để tương thích với code của bạn.
        Input: flat_content (str)
        Output: flat_content (str), final_refs (list)
        """
        print(f"   🔍 Scanning references for {self.version} using Hybrid Parser...")
        
        # --- BƯỚC 1: PHÂN TÍCH CẤU TRÚC BẰNG TEXSOUP ---
        # TexSoup giúp tìm file và citation chính xác, bỏ qua comment
        try:
            soup = TexSoup(flat_content)
        except Exception as e:
            print(f"      ⚠️ TexSoup parsing failed (syntax error?): {e}")
            # Nếu file lỗi quá nặng, trả về rỗng hoặc bạn có thể fallback về regex cũ ở đây
            return flat_content, []

        # 1.1: Tìm danh sách các key ĐƯỢC CITE trong bài (Usage Filter)
        used_keys = set()
        # Các lệnh cite phổ biến: \cite, \citet, \citep, \nocite...
        citation_cmds = soup.find_all(['cite', 'citet', 'citep', 'nocite', 'citeauthor', 'citeyear'])
        
        for cmd in citation_cmds:
            # Lấy argument cuối cùng (thường là list keys)
            if cmd.args:
                # TexSoup node -> string
                key_str = cmd.args[-1].string
                if key_str:
                    for k in key_str.split(','):
                        used_keys.add(k.strip())
        
        # 1.2: Tìm và Parse các file nguồn (.bib / .bbl)
        # Tìm lệnh \bibliography{...} hoặc \addbibresource{...}
        bib_cmds = soup.find_all(['bibliography', 'addbibresource'])
        processed_files = set()

        for cmd in bib_cmds:
            if cmd.args:
                filenames = cmd.args[-1].string
                if filenames:
                    for fname in filenames.split(','):
                        fname = fname.strip()
                        if not fname or fname in processed_files: continue
                        
                        # Logic: Ưu tiên .bib -> Nếu không có thì tìm .bbl
                        if self._try_parse_bib(fname):
                            processed_files.add(fname)
                        elif self._try_parse_bbl(fname):
                            processed_files.add(fname)

        # 1.3: Tìm và Parse Embedded Reference (Viết thẳng trong .tex)
        embedded_envs = soup.find_all('thebibliography')
        for env in embedded_envs:
            # Chuyển node TexSoup thành string để regex xử lý bibitem nhanh hơn
            self._parse_bibitem_content(str(env), source_type="embedded")

        # --- BƯỚC 2: FILTER & FORMAT OUTPUT ---
        # Chuyển từ Dict nội bộ -> List kết quả (chỉ lấy những ref được cite)
        final_refs = []
        skipped_count = 0
        
        # Check usage: Nếu key nằm trong used_keys hoặc dùng \nocite{*}
        is_wildcard = '*' in used_keys
        
        for key, ref_obj in self.raw_refs.items():
            if is_wildcard or key in used_keys:
                final_refs.append(ref_obj)
            else:
                skipped_count += 1
                
        print(f"      Found {len(self.raw_refs)} raw refs. Kept {len(final_refs)} unique refs (Skipped {skipped_count}).")
        
        return flat_content, final_refs

    def _try_parse_bib(self, filename):
        """Xử lý file .bib (dùng bibtexparser)"""
        # Fix extension
        if not filename.lower().endswith('.bib'): filename += '.bib'
        path = os.path.join(self.root_dir, filename)
        
        if not os.path.exists(path): return False
        
        print(f"      📖 Reading .bib file (Lib): {filename}")
        try:
            with open(path, 'r', encoding='utf-8', errors='ignore') as f:
                # Cấu hình parser
                parser = BibTexParser(common_strings=True)
                parser.ignore_nonstandard_types = True 
                parser.homogenise_fields = False
                
                db = bibtexparser.load(f, parser=parser)
            
            count = 0
            for entry in db.entries:
                key = entry.get('ID', '').strip()
                if not key: continue

                # Lưu vào dict raw_refs
                self.raw_refs[key] = {
                    "key": key,
                    "raw_text": self._dict_to_bibtex_string(entry),
                    "type": f"bib_{entry.get('ENTRYTYPE', 'misc').lower()}",
                    "parsed_fields": entry,
                    "source": filename
                }
                count += 1
                
            print(f"      -> Parsed {count} entries from .bib")
            return True
        except Exception as e:
            print(f"      ❌ Error parsing .bib: {e}")
            return False

    def _try_parse_bbl(self, filename):
        """Xử lý file .bbl (dùng Regex + Sentinel)"""
        # Fix extension (đảm bảo tìm đúng file .bbl)
        base_name = os.path.splitext(filename)[0]
        path = os.path.join(self.root_dir, base_name + '.bbl')
        
        if not os.path.exists(path): return False
        
        print(f"      📖 Reading .bbl file: {base_name}.bbl")
        try:
            with open(path, 'r', encoding='utf-8', errors='ignore') as f:
                content = f.read()
            
            count_before = len(self.raw_refs)
            self._parse_bibitem_content(content, source_type=f"{base_name}.bbl")
            print(f"      -> Parsed {len(self.raw_refs) - count_before} new entries from .bbl")
            return True
        except Exception as e:
            print(f"      ❌ Error parsing .bbl: {e}")
            return False

    def _parse_bibitem_content(self, text, source_type="bibitem"):
        """
        Hàm parse lõi cho \bibitem. 
        Sử dụng kỹ thuật Sentinel để đảm bảo Regex luôn bắt được item cuối cùng.
        """
        # --- FIX QUAN TRỌNG: SENTINEL TRICK ---
        text += "\n\\bibitem{SENTINEL_MARKER_FIX}"
        
        matches = self.REGEX_BIBITEM.findall(text)
        
        for label, key, content in matches:
            clean_key = key.strip()
            
            # Bỏ qua item giả
            if clean_key == "SENTINEL_MARKER_FIX": continue
                
            # Clean content
            clean_content = re.sub(r'\s+', ' ', content).strip()
            
            # Chỉ lưu nếu chưa tồn tại (Ưu tiên .bib đã parse trước đó nếu có trùng key)
            if clean_key not in self.raw_refs:
                self.raw_refs[clean_key] = {
                    "key": clean_key,
                    "raw_text": clean_content,
                    "type": "bibitem",
                    "label": label.strip() if label else None,
                    "source": source_type
                }

    def _dict_to_bibtex_string(self, entry):
        """Helper tái tạo string từ object bibtexparser"""
        lines = [f"@{entry.get('ENTRYTYPE', 'misc')}{{{entry.get('ID', '')},"]
        for k, v in entry.items():
            if k in ['ENTRYTYPE', 'ID']: continue
            lines.append(f"  {k} = {{{v}}},")
        lines.append("}")
        return "\n".join(lines)

In [50]:
import os
import re
import bibtexparser
from bibtexparser.bparser import BibTexParser
from TexSoup import TexSoup

class ReferenceParser:
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.raw_refs = {} # Dùng Dict để tự động khử trùng lặp theo Key
        
        # Regex chuyên dụng để quét nội dung của \bibitem (cho .bbl và embedded)
        # Lưu ý: Lookahead (?=...) sẽ hoạt động hoàn hảo nhờ kỹ thuật Sentinel thêm vào sau.
        self.REGEX_BIBITEM_CONTENT = re.compile(
            r'\\bibitem\s*(?:\[(.*?)\])?\s*\{(.*?)\}\s*(.*?)(?=(?:\\bibitem)|(?:\\end\{thebibliography\})|\Z)', 
            re.DOTALL
        )

    def parse(self, flat_tex_content):
        """
        Hàm chính: Nhận vào nội dung file .tex đã được flatten.
        Trả về: (Danh sách key được cite, Danh sách object reference chi tiết)
        """
        print("🚀 Starting Reference Parser...")
        
        # --- BƯỚC 1: Dùng TexSoup để phân tích cấu trúc file .tex ---
        # TexSoup tự động bỏ qua các dòng comment (%) -> Dữ liệu sạch
        try:
            soup = TexSoup(flat_tex_content)
        except Exception as e:
            print(f"⚠️ TexSoup failed to parse main content (using raw fallback): {e}")
            return [], [] # Hoặc fallback sang regex thuần nếu cần

        # 1.1 Tìm các citations (để biết key nào thực sự được dùng)
        used_keys = set()
        # Các lệnh cite phổ biến: \cite, \citet, \citep, ...
        citation_cmds = soup.find_all(['cite', 'citet', 'citep', 'nocite'])
        for cmd in citation_cmds:
            # Lấy argument cuối cùng (thường là list keys)
            if cmd.args:
                key_str = cmd.args[-1].string
                if key_str:
                    for k in key_str.split(','):
                        used_keys.add(k.strip())
        
        print(f"ℹ️ Found {len(used_keys)} unique citations in text.")

        # --- BƯỚC 2: Xử lý 3 trường hợp nguồn Reference ---

        # TRƯỜNG HỢP 1: File .bib (External)
        # Tìm lệnh \bibliography{file} hoặc \addbibresource{file}
        bib_cmds = soup.find_all(['bibliography', 'addbibresource'])
        for cmd in bib_cmds:
            if cmd.args:
                filenames = cmd.args[-1].string
                if filenames:
                    for fname in filenames.split(','):
                        self._process_external_bib(fname.strip())

        # TRƯỜNG HỢP 2: File .bbl (External - Thường đi kèm nếu ko có .bib)
        # Logic: Nếu tìm thấy file .bbl trùng tên với file .tex chính hoặc được include
        # (Thường ta quét thư mục để tìm file .bbl tương ứng nếu .bib không tồn tại)
        # Ở đây giả sử ta quét các file được khai báo trong bibliography nhưng đuôi là .bbl 
        # hoặc tự động check file cùng tên.
        # (Bạn có thể thêm logic quét thư mục tại đây nếu cần)
        
        # TRƯỜNG HỢP 3: Embedded (Viết thẳng trong .tex)
        # Tìm môi trường \begin{thebibliography}
        embedded_envs = soup.find_all('thebibliography')
        for env in embedded_envs:
            print("ℹ️ Found embedded 'thebibliography' environment.")
            # Convert node TexSoup ngược lại thành string để dùng Regex parse bibitem
            # Lý do: TexSoup parse bibitem item đôi khi bị chậm/lỗi nếu text rác nhiều.
            self._parse_bibitem_content(str(env), source="embedded")

        # --- BƯỚC 3: LỌC KẾT QUẢ ---
        final_results = []
        for key, ref_obj in self.raw_refs.items():
            # Nếu key nằm trong list cite HOẶC key là nocite (*)
            if key in used_keys or '*' in used_keys:
                final_results.append(ref_obj)
        
        print(f"✅ Finished. Captured {len(final_results)} references.")
        return list(used_keys), final_results

    def _process_external_bib(self, filename):
        """Xử lý file .bib (dùng bibtexparser)"""
        # Fix extension
        if not filename.endswith('.bib'): 
            # Check xem file .bib hay .bbl tồn tại
            if os.path.exists(os.path.join(self.root_dir, filename + '.bib')):
                filename += '.bib'
            elif os.path.exists(os.path.join(self.root_dir, filename + '.bbl')):
                # Nếu chỉ có .bbl -> Chuyển sang xử lý bbl
                return self._process_external_bbl(filename + '.bbl')
            else:
                return # File không tồn tại

        path = os.path.join(self.root_dir, filename)
        print(f"   📖 Parsing .bib file: {filename}")
        
        try:
            with open(path, 'r', encoding='utf-8', errors='ignore') as f:
                # Cấu hình parser cho phép lỗi nhẹ
                parser = BibTexParser(common_strings=True)
                parser.ignore_nonstandard_types = True
                parser.homogenise_fields = False
                
                db = bibtexparser.load(f, parser=parser)
                
                for entry in db.entries:
                    key = entry.get('ID')
                    if key:
                        self.raw_refs[key] = {
                            'key': key,
                            'type': entry.get('ENTRYTYPE'),
                            'raw_text': str(entry), # Hoặc build lại string từ dict
                            'data': entry, # Dữ liệu cấu trúc sạch
                            'source_file': filename
                        }
        except Exception as e:
            print(f"   ❌ Error parsing .bib: {e}")

    def _process_external_bbl(self, filename):
        """Xử lý file .bbl (dùng Regex + Sentinel)"""
        path = os.path.join(self.root_dir, filename)
        print(f"   📖 Parsing .bbl file: {filename}")
        
        if os.path.exists(path):
            with open(path, 'r', encoding='utf-8', errors='ignore') as f:
                content = f.read()
                self._parse_bibitem_content(content, source=filename)

    def _parse_bibitem_content(self, text_content, source="unknown"):
        """
        Core function để parse nội dung dạng \bibitem{...} ...
        Dùng kỹ thuật Sentinel để đảm bảo lấy được item cuối cùng.
        """
        # --- SENTINEL TRICK ---
        # Thêm một item giả vào cuối chuỗi. 
        # Điều này giúp Regex Lookahead (?=\\bibitem) luôn đúng cho item thật cuối cùng.
        text_content += "\n\\bibitem{__SENTINEL_END__}"
        
        matches = self.REGEX_BIBITEM_CONTENT.findall(text_content)
        
        for label, key, content in matches:
            key = key.strip()
            if key == "__SENTINEL_END__": continue # Bỏ qua lính canh
            
            # Làm sạch content (xóa xuống dòng thừa)
            clean_content = re.sub(r'\s+', ' ', content).strip()
            
            # Chỉ thêm nếu chưa tồn tại (ưu tiên .bib nếu đã parse trước đó)
            if key not in self.raw_refs:
                self.raw_refs[key] = {
                    'key': key,
                    'type': 'bibitem',
                    'raw_text': clean_content,
                    'label': label.strip() if label else None,
                    'source_file': source
                }

# --- CÁCH SỬ DỤNG ---
# parser = ReferenceParser(root_dir="./my_paper_folder")
#
# # Đọc file tex chính (giả sử bạn đã flatten nó rồi)
# with open("./my_paper_folder/main_flat.tex", "r", encoding="utf-8") as f:
#     flat_content = f.read()
#
# used_keys, refs = parser.parse(flat_content)
#
# for r in refs:
#     print(f"[{r['key']}] -> {r.get('raw_text')[:50]}...")

In [51]:
# Configuration
DATA_RAW_PATH = os.path.abspath(os.path.join(os.getcwd(), '..', 'data_raw'))

print(f"Data raw path: {DATA_RAW_PATH}")

if not os.path.exists(DATA_RAW_PATH):
    print("❌ Warning: Data raw path does not exist. Please check the path.")
else:
    print("✅ Data raw path found.")

Data raw path: d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\data_raw
✅ Data raw path found.


In [52]:
paper_base_id = '2403-00531'
versions_to_process = '2403-00531v2'


In [53]:
# %%
# --- TEST RUNNER CHO REFERENCE PROCESSOR ---

# 1. Cấu hình đường dẫn cho Version cụ thể
ver_path = os.path.join(DATA_RAW_PATH, paper_base_id, 'tex', versions_to_process)
print(f"📂 Testing directory: {ver_path}")

# 2. Bước tiên quyết: Flatten nội dung trước (vì ReferenceProcessor cần text phẳng)
root_file = find_root_tex_file(ver_path)

if not root_file:
    print("❌ Error: Could not find root tex file.")
else:
    print(f"✅ Root file found: {os.path.basename(root_file)}")
    
    # Flatten
    flattener = LatexFlattener(root_file, paper_id=paper_base_id, version=versions_to_process, remove_references=False)
    flat_result = flattener.flatten()
    flat_content = flat_result['content']
    print(f"📄 Flattened content size: {len(flat_content)} characters")
    print('\\bibliography{apssamp}' in flat_content)
# \bibliography{apssamp}% Produces the bibliography via BibTeX.
# 
    # 3. Khởi tạo và chạy ReferenceProcessor
    print("\n--- START REFERENCE PROCESSING ---")
    print(ver_path)
    ref_processor = ReferenceProcessor(paper_base_id, versions_to_process, ver_path)
    
    # Hàm này sẽ in ra log chi tiết về việc tìm thấy .bbl hay .bib và số lượng ref lọc được
    content_after_process, final_refs = ref_processor.process_references(flat_content)
    
    print("--- END REFERENCE PROCESSING ---\n")

    # 4. Kiểm tra kết quả đầu ra
    print(f"📊 Total Valid References: {len(final_refs)}")
    
    if len(final_refs) > 0:
        print("\n🔎 Sample Reference 1:")
        print(json.dumps(final_refs[0], indent=2, ensure_ascii=False))
        
        # Kiểm tra xem key của ref có đúng format không
        print(f"\nKey: {final_refs[0]['key']}")
        print(f"Source Type: {final_refs[0]['type']}")
    else:
        print("⚠️ No references found. Check if the paper actually has citations or if regex needs adjustment.")

📂 Testing directory: d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\data_raw\2403-00531\tex\2403-00531v2
✅ Root file found: apssamp.tex
📝 Khởi tạo LatexFlattener cho Paper: 2403-00531, Version: 2403-00531v2
   Remove references: No
📄 Flattened content size: 72207 characters
True

--- START REFERENCE PROCESSING ---
d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\data_raw\2403-00531\tex\2403-00531v2
   🔍 Scanning references for 2403-00531v2 using Hybrid Parser...
      📖 Reading .bib file (Lib): apssamp.bib
      -> Parsed 67 entries from .bib
      Found 67 raw refs. Kept 67 unique refs (Skipped 0).
--- END REFERENCE PROCESSING ---

📊 Total Valid References: 67

🔎 Sample Reference 1:
{
  "key": "Lelli:2016zqa",
  "raw_text": "@article{Lelli:2016zqa,\n  year = {2016},\n  pages = {157},\n  volume = {152},\n  journal = {Astron. J.},\n  doi = {10.3847/0004-6256/152/6/157},\n  primaryclass = {astro-ph.GA},\n  archiveprefix = {arXiv},\n  eprint = {1606.09251},\n  

In [54]:
with open(f'references-{paper_base_id}-v2.json', 'w', encoding = 'utf-8') as f:
    json.dump(final_refs, f, indent=2, ensure_ascii=False)

In [55]:

print(flat_content)





















\documentclass[
 reprint,










 amsmath,amssymb,
 aps,
pra,




floatfix,
]{revtex4-2}

\usepackage{graphicx}

\usepackage{csvsimple}
\usepackage{dcolumn}
\usepackage{array} 


\usepackage{bm}
\usepackage[dvipsnames]{xcolor}

\usepackage{subcaption}
\usepackage{multirow}
\usepackage{times}
\usepackage{placeins}
\usepackage{hyperref}
\usepackage{booktabs}
\bibliographystyle{apsrev4-1}

\begin{document}
\hbadness=99999

\preprint{APS/123-QED}

\title{Phenomenology of renormalization group improved gravity from the kinematics of SPARC galaxies.}


\author{Esha Bhatia}
\email{b.esha@iitg.ac.in}
\author{Sayan Chakrabarti}
\email{sayan.chakrabarti@iitg.ac.in}
\author{Sovan Chakraborty}
\email{sovan@iitg.ac.in}



\affiliation{Department of Physics, Indian Institute of Technology, Guwahati 781039, India}




\date{\today}
             

\begin{abstract}
Renormalization Group correction to General Relativity (RGGR) proposes a logarithmic running of the gravitational co

In [56]:
import hashlib
import re

class ReferenceDeduplicator:
    def __init__(self):
        # Kho chứa reference duy nhất: { fingerprint: {data} }
        self.unique_refs_pool = {} 
        
        # Danh sách key chuẩn theo thứ tự: ['ref_0', 'ref_1', ...]
        self.canonical_keys = []
        
        # Map để thay thế trong text: { version: { old_key: new_key } }
        self.version_maps = {}

    def _create_fingerprint(self, text):
        """
        Tạo dấu vân tay từ nội dung reference.
        Chỉ giữ lại chữ cái và số, bỏ qua dấu câu và khoảng trắng để so sánh chính xác.
        """
        # Xóa mọi thứ không phải chữ/số và chuyển về chữ thường
        clean_text = re.sub(r'[^a-z0-9]', '', text.lower())
        return hashlib.md5(clean_text.encode('utf-8')).hexdigest()

    def add_references(self, version, refs_list):
        """
        Nhận danh sách refs từ một version và xử lý dedup.
        """
        print(f"   🧩 Deduplicating refs for {version}...")
        self.version_maps[version] = {}
        
        for ref in refs_list:
            old_key = ref['key']
            raw_text = ref['raw_text']
            
            # 1. Tạo fingerprint
            fp = self._create_fingerprint(raw_text)
            
            canonical_key = None
            
            # 2. Kiểm tra trùng lặp
            if fp in self.unique_refs_pool:
                # Đã tồn tại -> Lấy key chuẩn cũ
                canonical_key = self.unique_refs_pool[fp]['canonical_key']
                
                # Unionize: Cập nhật thông tin nếu bản mới đầy đủ hơn bản cũ (Optional)
                # Ví dụ: Nếu bản cũ ngắn quá, lấy bản mới dài hơn làm raw_text chính
                if len(raw_text) > len(self.unique_refs_pool[fp]['raw_text']):
                     self.unique_refs_pool[fp]['raw_text'] = raw_text
                     
            else:
                # Chưa tồn tại -> Tạo mới
                new_idx = len(self.canonical_keys)
                canonical_key = f"ref_{new_idx}" # Tạo key chuẩn: ref_0, ref_1...
                
                self.unique_refs_pool[fp] = {
                    'canonical_key': canonical_key,
                    'raw_text': raw_text,
                    'original_refs': [] # Để trace lại nếu cần
                }
                self.canonical_keys.append(canonical_key)
            
            # 3. Lưu mapping cho version này (Old Key -> Canonical Key)
            # Chỉ lưu nếu key khác nhau để tối ưu hiệu năng replace
            if old_key != canonical_key:
                self.version_maps[version][old_key] = canonical_key

    def get_replacements(self, version):
        """Trả về dict {old: new} để replace trong text của version đó"""
        return self.version_maps.get(version, {})

    def export_bib_string(self):
        """Xuất chuỗi BibTeX chuẩn cho file refs.bib"""
        bib_content = ""
        # Duyệt theo thứ tự canonical keys để file đẹp
        for fp in self.unique_refs_pool:
            item = self.unique_refs_pool[fp]
            key = item['canonical_key']
            text = item['raw_text']
            
            # Tạo entry @misc đơn giản chứa full text
            # Vì ta trích xuất từ \bibitem nên không có field tách biệt, tống hết vào title/note
            entry = f"@misc{{{key},\n  text = {{{text}}}\n}}\n\n"
            bib_content += entry
            
        return bib_content

def replace_citations_in_text(text, replacement_map):
    """
    Thay thế \cite{old} thành \cite{new} trong toàn bộ văn bản.
    """
    if not replacement_map:
        return text
        
    # Cách đơn giản: Replace chuỗi. 
    # An toàn hơn: Dùng Regex với hàm callback để chỉ replace TRONG lệnh \cite
    # Nhưng với bài này, replace chuỗi key thường đủ an toàn vì key bibtex ít trùng từ thường.
    
    # Để an toàn tối đa, ta dùng Regex Callback:
    def replace_match(match):
        # match.group(0) là toàn bộ \cite{...}
        # match.group(1) là nội dung bên trong {}
        inner = match.group(1)
        keys = [k.strip() for k in inner.split(',')]
        
        new_keys = []
        for k in keys:
            # Nếu key có trong map thì đổi, không thì giữ nguyên
            new_keys.append(replacement_map.get(k, k))
            
        return f"\\cite{{{', '.join(new_keys)}}}"

    # Regex bắt \cite{...}
    pattern = re.compile(r'(\\cite[a-z]*\s*(?:\[.*?\])?\s*\{)([^}]+)(\})', re.IGNORECASE)
    
    # group 1: \cite{
    # group 2: key1, key2
    # group 3: }
    return pattern.sub(replace_match, text)

In [57]:
# 1. Setup
paper_base_id = '2403-00531'
versions = ['2403-00531v1', '2403-00531v2'] # List các version
ref_deduplicator = ReferenceDeduplicator()
content_deduplicator = ContentDeduplicator()

# Lưu trữ nội dung phẳng tạm thời để parse sau khi dedup ref
temp_flat_contents = {} 

print("🚀 STEP 1: GATHERING & REFERENCE EXTRACTION")
for ver in versions:
    ver_path = os.path.join(DATA_RAW_PATH, paper_base_id, 'tex', ver)
    
    # A. Flatten
    root_file = find_root_tex_file(ver_path)
    if not root_file: continue
    
    flattener = LatexFlattener(root_file, paper_id=paper_base_id, version=ver,remove_references=False)
    flat_res = flattener.flatten()
    flat_text = flat_res['content']
    
    # B. Reference Extract (Dùng class mới bạn đã test)
    ref_proc = ReferenceProcessor(paper_base_id, ver, ver_path)
    flat_text, refs = ref_proc.process_references(flat_text)
    
    # C. Add to Dedup
    ref_deduplicator.add_references(ver, refs)
    
    # Lưu lại text gốc để tí nữa sửa
    temp_flat_contents[ver] = flat_text

print("\n🚀 STEP 2: TEXT REPLACEMENT & PARSING")
for ver in versions:
    if ver not in temp_flat_contents: continue
    
    # A. Lấy map thay thế (Old -> New)
    replacements = ref_deduplicator.get_replacements(ver)
    raw_text = temp_flat_contents[ver]
    
    # B. Sửa Key trong Text
    # Hàm này sẽ đổi \cite{hinton06} thành \cite{ref_0}
    updated_text = replace_citations_in_text(raw_text, replacements)
    
    print(f"   📝 Parsed {ver} with {len(replacements)} citation updates.")
    
    # C. Parse Structure (Dùng text ĐÃ SỬA)
    builder = LatexStructureBuilder(updated_text, paper_base_id, ver)
    root_node = builder.build_coarse_tree()
    
    # D. Clean Content
    processor = LatexContentProcessor(paper_base_id, ver)
    processor.process_tree(root_node)
    
    # E. Content Dedup
    content_deduplicator.process_version(ver, root_node)

print("\n🚀 STEP 3: EXPORT")

# 1. Export refs.bib
bib_str = ref_deduplicator.export_bib_string()
with open('refs.bib', 'w', encoding='utf-8') as f:
    f.write(bib_str)
print("✅ Exported refs.bib")

# 2. Export hierarchy.json
final_hier = content_deduplicator.get_final_json()
with open('hierarchy.json', 'w', encoding='utf-8') as f:
    json.dump(final_hier, f, indent=2, ensure_ascii=False)
print("✅ Exported hierarchy.json")

🚀 STEP 1: GATHERING & REFERENCE EXTRACTION
📝 Khởi tạo LatexFlattener cho Paper: 2403-00531, Version: 2403-00531v1
   Remove references: No
   🔍 Scanning references for 2403-00531v1 using Hybrid Parser...
      📖 Reading .bib file (Lib): ref.bib
      -> Parsed 54 entries from .bib
      Found 54 raw refs. Kept 49 unique refs (Skipped 5).
   🧩 Deduplicating refs for 2403-00531v1...
📝 Khởi tạo LatexFlattener cho Paper: 2403-00531, Version: 2403-00531v2
   Remove references: No
   🔍 Scanning references for 2403-00531v2 using Hybrid Parser...
      📖 Reading .bib file (Lib): apssamp.bib
      -> Parsed 67 entries from .bib
      Found 67 raw refs. Kept 67 unique refs (Skipped 0).
   🧩 Deduplicating refs for 2403-00531v2...

🚀 STEP 2: TEXT REPLACEMENT & PARSING
   📝 Parsed 2403-00531v1 with 49 citation updates.
🔍 Xử lý Preamble để trích xuất Title, Authors, Abstract...
🔄 Processing hierarchy for version: 1 (from 2403-00531v1)
   📝 Parsed 2403-00531v2 with 67 citation updates.
🔍 Xử lý Preamb